In [1]:
# %load lowlight_train.py
# 对其进行改进，使其能训练2Dtif图像的模型

import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import datetime
import dataloader
import model
import Myloss
import numpy as np
from torchvision import transforms

torch.set_default_dtype(torch.float64)


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


def train(config):

    os.environ['CUDA_VISIBLE_DEVICES']='0'

    DCE_net = model.enhance_net_nopool().cuda()
    
    DCE_net.apply(weights_init)
#     if config.load_pretrain == True:
#         DCE_net.load_state_dict(torch.load(config.pretrain_dir))
        
    print(config.num_workers)
    train_dataset = dataloader.lowlight_loader(config.lowlight_images_path)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.train_batch_size, 
                                               shuffle=True, num_workers=config.num_workers, 
                                               pin_memory=True)
#     L_color = Myloss.L_color()
    L_spa = Myloss.L_spa()
    L_exp = Myloss.L_exp(16,0.6)
    L_TV = Myloss.L_TV()
    
    optimizer = torch.optim.Adam(DCE_net.parameters(), lr=config.lr, weight_decay=config.weight_decay)

    DCE_net.train()
    
    loss_list = []
    for epoch in range(config.num_epochs):
        print('---------------第{} epochs-------------'.format(epoch))
        for iteration, img_lowlight in enumerate(train_loader):
            img_lowlight = img_lowlight.cuda()

            enhanced_image_1,enhanced_image,A  = DCE_net(img_lowlight)

            Loss_TV = L_TV(A)
            loss_spa = torch.mean(L_spa(enhanced_image, img_lowlight))
#             loss_col = 5*torch.mean(L_color(enhanced_image))
            loss_exp = torch.mean(L_exp(enhanced_image))

            # best_loss
            loss =  Loss_TV + loss_spa + loss_exp
            loss_list.append([Loss_TV, loss_spa, loss_exp])
            
            optimizer.zero_grad()
            loss.backward()
#             torch.nn.utils.clip_grad_norm(DCE_net.parameters(),config.grad_clip_norm)
            optimizer.step()

            if ((iteration+1) % config.display_iter) == 0:
                print("Loss at iteration", iteration+1, ":", loss.item(),
                      '  Loss_TV:{} + loss_spa:{} + loss_exp:{}'.format(Loss_TV , loss_spa , loss_exp))
            if ((iteration+1) % config.snapshot_iter) == 0:
                torch.save(DCE_net.state_dict(), config.snapshots_folder 
                           +datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S_')
                           +"Epoch" + str(epoch) + '.pth')
            
        np.save('data/loss/epochs_{}'.format(epoch+1), loss_list)



if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # Input Parameters
    parser.add_argument('--lowlight_images_path', type=str, default="E:/zero_dce_data/max_projection_img/")
    parser.add_argument('--lr', type=float, default=0.001)
    parser.add_argument('--weight_decay', type=float, default=0.0001)
    parser.add_argument('--grad_clip_norm', type=float, default=0.1)
    parser.add_argument('--num_epochs', type=int, default=200)   # 200
    parser.add_argument('--train_batch_size', type=int, default=64)    # 8
    parser.add_argument('--val_batch_size', type=int, default=4)   # 4
    parser.add_argument('--num_workers', type=int, default=1)         # 4
    parser.add_argument('--display_iter', type=int, default=10)       # 10
    parser.add_argument('--snapshot_iter', type=int, default=10)      # 10
    parser.add_argument('--snapshots_folder', type=str, default="snapshots/")
    parser.add_argument('--load_pretrain', type=bool, default= False)    # False
    parser.add_argument('--pretrain_dir', type=str, default= "snapshots/Epoch99.pth")
    parser.add_argument('--h', type=str, default= "snapshots/Epoch99.pth")
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

    config = parser.parse_args()

    if not os.path.exists(config.snapshots_folder):
        os.mkdir(config.snapshots_folder)


    train(config)




1
Total training examples: 3440
---------------第0 epochs-------------


E:\anaconda\envs\pytorch03\lib\site-packages\torch\nn\functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Loss at iteration 10 : 0.3517342101358226   Loss_TV:0.0016647892502890662 + loss_spa:0.0007527431292959933 + loss_exp:0.34931667775623754
Loss at iteration 20 : 0.33920161138279986   Loss_TV:0.014190103482445 + loss_spa:0.017702036852783824 + loss_exp:0.30730947104757106
Loss at iteration 30 : 0.23358479247267863   Loss_TV:8.151276579063573e-07 + loss_spa:0.051114711703076965 + loss_exp:0.18246926564194377
Loss at iteration 40 : 0.24226768798353182   Loss_TV:1.3146127107017776e-23 + loss_spa:0.06507504876630578 + loss_exp:0.17719263921722606
Loss at iteration 50 : 0.2332744489790916   Loss_TV:4.739000847409537e-38 + loss_spa:0.05734769271968278 + loss_exp:0.17592675625940885
---------------第1 epochs-------------
Loss at iteration 10 : 0.24069476942377366   Loss_TV:0.0 + loss_spa:0.06153314479063576 + loss_exp:0.1791616246331379
Loss at iteration 20 : 0.2387480626849367   Loss_TV:0.0 + loss_spa:0.05923464102914777 + loss_exp:0.17951342165578893
Loss at iteration 30 : 0.22913631769745813

Loss at iteration 40 : 0.2480420647875458   Loss_TV:2.995386766158313e-06 + loss_spa:0.0686324839065554 + loss_exp:0.17940658549422428
Loss at iteration 50 : 0.23417632550524878   Loss_TV:3.5656809449942646e-06 + loss_spa:0.061632364354812244 + loss_exp:0.17254039546949154
---------------第12 epochs-------------
Loss at iteration 10 : 0.23093725015119904   Loss_TV:4.2954595270533724e-06 + loss_spa:0.06609878683651049 + loss_exp:0.16483416785516147
Loss at iteration 20 : 0.23873392642074517   Loss_TV:4.715528391123864e-06 + loss_spa:0.051962477258205735 + loss_exp:0.1867667336341483
Loss at iteration 30 : 0.24279920924614157   Loss_TV:5.088638687450251e-06 + loss_spa:0.058436120879722185 + loss_exp:0.18435799972773192
Loss at iteration 40 : 0.23340843463784355   Loss_TV:5.366643558024122e-06 + loss_spa:0.05827909764139934 + loss_exp:0.17512397035288618
Loss at iteration 50 : 0.2327076478805408   Loss_TV:5.622932956444682e-06 + loss_spa:0.06310747171710873 + loss_exp:0.16959455323047562
-

---------------第23 epochs-------------
Loss at iteration 10 : 0.23112787753894046   Loss_TV:6.3931967623996995e-06 + loss_spa:0.0552652496935892 + loss_exp:0.17585623464858885
Loss at iteration 20 : 0.23859977694114165   Loss_TV:6.4112650677961825e-06 + loss_spa:0.06160276565847689 + loss_exp:0.17699060001759698
Loss at iteration 30 : 0.2293172742234601   Loss_TV:6.3322739838227634e-06 + loss_spa:0.06396950708106394 + loss_exp:0.16534143486841232
Loss at iteration 40 : 0.24243944176159438   Loss_TV:6.308508594043551e-06 + loss_spa:0.05455365659786169 + loss_exp:0.18787947665513866
Loss at iteration 50 : 0.23827750671475215   Loss_TV:6.1109218755077675e-06 + loss_spa:0.06320257838908593 + loss_exp:0.1750688174037907
---------------第24 epochs-------------
Loss at iteration 10 : 0.23640754317014384   Loss_TV:5.935728875608865e-06 + loss_spa:0.06910194840214086 + loss_exp:0.16729965903912736
Loss at iteration 20 : 0.2316562355780924   Loss_TV:5.978917027698588e-06 + loss_spa:0.056809474522

...

Loss at iteration 30 : 0.24396551659488233   Loss_TV:6.439795803453081e-06 + loss_spa:0.05452869239969291 + loss_exp:0.18943038439938598
Loss at iteration 40 : 0.2369380888501936   Loss_TV:5.907035821837758e-06 + loss_spa:0.05863588706136616 + loss_exp:0.1782962947530056
Loss at iteration 50 : 0.24281531166898682   Loss_TV:6.334872326553659e-06 + loss_spa:0.0536130989770283 + loss_exp:0.18919587781963196
---------------第149 epochs-------------
Loss at iteration 10 : 0.23024589981194818   Loss_TV:6.828400176996676e-06 + loss_spa:0.05153157649478585 + loss_exp:0.17870749491698534
Loss at iteration 20 : 0.2333148981532375   Loss_TV:6.520006598835548e-06 + loss_spa:0.05475692271892581 + loss_exp:0.17855145542771286
Loss at iteration 30 : 0.24287776379056228   Loss_TV:6.057617293166489e-06 + loss_spa:0.05864094224728841 + loss_exp:0.1842307639259807
Loss at iteration 40 : 0.23782418128309668   Loss_TV:6.560450330551827e-06 + loss_spa:0.056851896230757454 + loss_exp:0.18096572460200866
Loss 

Loss at iteration 50 : 0.24353253859003254   Loss_TV:7.117205256474184e-06 + loss_spa:0.05071528719778208 + loss_exp:0.192810134186994
---------------第160 epochs-------------
Loss at iteration 10 : 0.2349247721746851   Loss_TV:7.078716553871799e-06 + loss_spa:0.06280410541139493 + loss_exp:0.1721135880467363
Loss at iteration 20 : 0.23106753073150701   Loss_TV:5.504468689423993e-06 + loss_spa:0.04974862850798063 + loss_exp:0.18131339775483696
Loss at iteration 30 : 0.22882542977270454   Loss_TV:6.858037856816773e-06 + loss_spa:0.062457047880642505 + loss_exp:0.16636152385420522
Loss at iteration 40 : 0.23439811827310958   Loss_TV:6.128345662380817e-06 + loss_spa:0.0504290621522049 + loss_exp:0.1839629277752423
Loss at iteration 50 : 0.2467150737454588   Loss_TV:6.91904381140417e-06 + loss_spa:0.06260582529727138 + loss_exp:0.18410232940437604
---------------第161 epochs-------------
Loss at iteration 10 : 0.2266593659144465   Loss_TV:6.87628827541993e-06 + loss_spa:0.05404778947562766 +

Loss at iteration 20 : 0.23736888264175043   Loss_TV:6.457818216282031e-06 + loss_spa:0.059738665505525576 + loss_exp:0.17762375931800858
Loss at iteration 30 : 0.24143695601411463   Loss_TV:6.714396611055589e-06 + loss_spa:0.06225901914184704 + loss_exp:0.17917122247565653
Loss at iteration 40 : 0.24009270559164592   Loss_TV:7.022395412895021e-06 + loss_spa:0.05258225829638004 + loss_exp:0.187503424899853
Loss at iteration 50 : 0.23340026073446402   Loss_TV:5.637157748362786e-06 + loss_spa:0.06875177738825311 + loss_exp:0.16464284618846256
---------------第172 epochs-------------
Loss at iteration 10 : 0.24556666363622776   Loss_TV:6.122569636288822e-06 + loss_spa:0.06057581513775777 + loss_exp:0.1849847259288337
Loss at iteration 20 : 0.2319112441914143   Loss_TV:6.643821365596395e-06 + loss_spa:0.04915703067556874 + loss_exp:0.18274756969447997
Loss at iteration 30 : 0.23497658079910466   Loss_TV:6.1383745387146935e-06 + loss_spa:0.053737113712979595 + loss_exp:0.18123332871158634
Lo

Loss at iteration 40 : 0.23105091321498528   Loss_TV:5.936482469384294e-06 + loss_spa:0.05497042989465971 + loss_exp:0.17607454683785617
Loss at iteration 50 : 0.23933679514536682   Loss_TV:7.048854270699924e-06 + loss_spa:0.06315329081891065 + loss_exp:0.17617645547218547
---------------第183 epochs-------------
Loss at iteration 10 : 0.23733956699018516   Loss_TV:5.714665295210855e-06 + loss_spa:0.05917923448847183 + loss_exp:0.17815461783641812
Loss at iteration 20 : 0.24592147188107702   Loss_TV:6.9518086165845425e-06 + loss_spa:0.04989120105893439 + loss_exp:0.19602331901352604
Loss at iteration 30 : 0.25008193497195363   Loss_TV:7.179874011005049e-06 + loss_spa:0.061563450510349914 + loss_exp:0.18851130458759274
Loss at iteration 40 : 0.23000983617411153   Loss_TV:5.852267071937757e-06 + loss_spa:0.053983414519414324 + loss_exp:0.17602056938762528
Loss at iteration 50 : 0.22740340975458684   Loss_TV:6.6091721098938e-06 + loss_spa:0.0629118038464115 + loss_exp:0.16448499673606543
-

---------------第194 epochs-------------
Loss at iteration 10 : 0.2400029236444703   Loss_TV:7.125000135685319e-06 + loss_spa:0.060123015864828544 + loss_exp:0.17987278277950605
Loss at iteration 20 : 0.242006275319641   Loss_TV:6.2873675075366006e-06 + loss_spa:0.05490009331161794 + loss_exp:0.18709989464051552
Loss at iteration 30 : 0.23573772313561386   Loss_TV:6.686991833412561e-06 + loss_spa:0.05292331102815519 + loss_exp:0.18280772511562526
Loss at iteration 40 : 0.22546238698657262   Loss_TV:6.265138286697175e-06 + loss_spa:0.06006498902736786 + loss_exp:0.16539113282091805
Loss at iteration 50 : 0.2377591261488982   Loss_TV:6.49055504752351e-06 + loss_spa:0.061742776436086645 + loss_exp:0.17600985915776401
---------------第195 epochs-------------
Loss at iteration 10 : 0.2355093085370511   Loss_TV:6.9056242749625676e-06 + loss_spa:0.07094877127244642 + loss_exp:0.1645536316403297
Loss at iteration 20 : 0.2257041807459692   Loss_TV:7.229242965513885e-06 + loss_spa:0.05993183574607

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

now = datetime.datetime.now()
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# 将当前时间转换为指定格式的字符串
time_str = now.strftime('%Y-%m-%d %H:%M:%S')

# 输出时间字符串
print(time_str)


2023-11-03 20:46:11
2023-11-03 20:46:11
